In [ ]:
!pip install qrcode
!pip install opencv-python
!pip install pillow
!pip install pycrypto
!pip install "pycryptodome==3.9.7"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 502.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp310-cp310-linux_x86_64.whl size=498545 sha256=9eb01762c6694f2e93f136c7e1525c2c2311341f6cdd7b36d61649e4471614a1
  Stored in directory: /root/.cache/pip/wheels/e8/4b/5b/b10a6fc885057b6ff9fbd5691d7e700d0a9408f80b7e6f12e0
Successfully built pycrypto
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 33.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycryptodome: filename=pycryptodome-3.9.7-cp310-cp310-linux_x86_64.whl size=13642429 sha256=8fbcdfd3bf039e05ba1b8f3cc16fd414c1610a5e3fa57b7d960fc8a03bb3c288
  Stored in directory: /root/.cache/pip/wheels/47/b6/b7/d78b614423573e0dac519eebfce754de45a158b7890d3c16f4
Success

In [ ]:
ACL_json = '''
[
    {
        "name":"user1",
        "resources":"resource1"
    },

    {
        "name":"user2",
        "resources":"resource2"
    },

    {
        "name":"user3",
        "resources":"resource3"
    }
]
'''

In [ ]:
import qrcode
import random
import string
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
import base64
import cv2 as cv
import json
import ast

# User requests access to a resource or service
user_name = "user2"
requested_resource = "resource1"
session_id = ''.join(random.choices(string.ascii_letters + string.digits, k=7))

key = RSA.generate(2048)
qr_code_data = {"user":user_name,"resource": requested_resource, "session_id": session_id}
qr_code_data_str = str(qr_code_data).encode('utf-8')
hash = SHA256.new(qr_code_data_str)
signature = pkcs1_15.new(key).sign(hash)

qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_H,
    box_size=10,
    border=4,
)
qr.add_data(qr_code_data_str)
qr.make(fit=True)
img = qr.make_image(fill_color="black", back_color="white").convert('RGB')
img.save("sample.png")


In [ ]:
import json
ACL = json.loads(ACL_json)
print(len(ACL))

3


In [ ]:
# verify the QR and user's identity
im = cv.imread('/content/sample.png')
det = cv.QRCodeDetector()
retval, points, straight_qrcode = det.detectAndDecode(im)
print(retval)
ret = ast.literal_eval(retval)
if ret["resource"] == requested_resource and ret["session_id"] == session_id:
  qr_code_data_str = str(retval).encode('utf-8')
  hash = SHA256.new(qr_code_data_str)
  public_key = key.publickey()
  try:
    pkcs1_15.new(public_key).verify(hash, signature)
  except (ValueError, TypeError):
    print("QR code verification failed. Access denied.")
    exit()
  for i in range(0,len(ACL)):
    temp = ACL[i]
    if(temp["name"]==ret["user"] and temp["resources"] == ret["resource"]):
      print("name : ",temp["name"], "resource : ",temp["resources"])
      print("Access granted")
      break
    else:
      print("Access denied.User is not authorized for this resource.")
else:
  print("QR code verification failed.Access denied.")



{'user': 'user2', 'resource': 'resource1', 'session_id': 'txALdCV'}
Access denied.User is not authorized for this resource.
Access denied.User is not authorized for this resource.
Access denied.User is not authorized for this resource.
